# Build Image

In this notebook, we show the following steps for deploying a web service using AML:

- Create an image
- Test image locally


In [ ]:
%matplotlib inline

import docker
import matplotlib.pyplot as plt
import numpy as np
import requests
from azureml.core import Workspace
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azureml._model_management._util import (get_docker_client, pull_docker_image)
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.image import ContainerImage
from dotenv import set_key, get_key, find_dotenv
from utilities import load_image_into_numpy_array
from PIL import Image

import sys
import os
import json
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
from matplotlib import pyplot as plt

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
image_name = 'maskrcnnoutofstockimage' # You can pick another image name here
model_name = get_key(env_path,"model_name")
resource_group = get_key(env_path, 'resource_group')

In [ ]:
set_key(env_path, "image_name", image_name)

## Get workspace
Load existing workspace from the config file info.

In [ ]:
ws = Workspace.from_config()
# print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Image

In [ ]:
# create yml file to be used in the image
conda_pack = ["tensorflow==1.12.0"]
requirements = ["Pillow==6.0.0", "azureml-defaults==1.0.23"]

imgenv = CondaDependencies.create(conda_packages=conda_pack,pip_packages=requirements)
with open("img_env.yml", "w") as f:
    f.write(imgenv.serialize_to_string())

In [ ]:
image_config = ContainerImage.image_configuration(execution_script = "detect.py",
                                                  runtime = "python",
                                                  conda_file = "img_env.yml",
                                                  description = "MaskRCNN image for out of stock model",
                                                  tags = {"data":"products","method":"tensorflow"}
                                               )

Configure Container for GPU Inference

In [ ]:
# create yml file to be used in the image
conda_pack = ["tensorflow-gpu==1.12.0"]
requirements = ["Pillow==6.0.0", "azureml-defaults==1.0.23"]

imgenv = CondaDependencies.create(conda_packages=conda_pack,pip_packages=requirements)
with open("img_env.yml", "w") as f:
    f.write(imgenv.serialize_to_string())

In [ ]:
image_config = ContainerImage.image_configuration(execution_script = "detect.py",
                                                  runtime = "python",
                                                  conda_file = "img_env.yml",
                                                  enable_gpu=True,
                                                  description = "MaskRCNN image for out of stock model",
                                                  tags = {"data":"products","method":"tensorflow"}
                                               )

In [ ]:
# create image. It may take upto 15-20 minutes. 
image = ContainerImage.create(name = image_name,
                              # this is the model object
                              models = [ws.models[model_name]],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

In [ ]:
# You can find the logs of image creation
# image.image_build_log_uri

# You can get the image object when not creating a new image
# image = ws.images[image_name]

## Test image locally
- Pull the image from ACR registry to local host 
- Start a container
- Test API call

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(reg_name))
print('Username: {}'.format(username))
print('Password: {}'.format(password))

In [ ]:
dc = get_docker_client()

In [ ]:
pull_docker_image(dc, image.image_location, username, password)

In [ ]:
# make sure port 80 is not occupied
container_labels = {'containerName': 'tfgpu'}
container = dc.containers.run(image.image_location, 
                                         detach=True, 
                                         ports={'5001/tcp': 80},
                                         labels=container_labels,
                                         runtime='nvidia' )

In [ ]:
for log_msg in container.logs(stream=True):
    str_msg = log_msg.decode('UTF8')
    print(str_msg)

In [ ]:
client = docker.APIClient()
details = client.inspect_container(container.id)

In [ ]:
service_ip = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostIp']
service_port = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostPort']

Wait a few seconds for the application to spin up and then check that everything works.

In [ ]:
print('Checking service on {} port {}'.format(service_ip, service_port))

In [ ]:
endpoint="http://__service_ip:__service_port"
endpoint = endpoint.replace('__service_ip', service_ip)
endpoint = endpoint.replace('__service_port', service_port)

In [ ]:
!curl 'http://{service_ip}:{service_port}/'

In [ ]:
img_data = open('IMAGE HERE', "rb")
%time r = requests.post('http://0.0.0.0:80/score', files={'image': img_data})

In [ ]:
print(r)
r.json()

In [ ]:
PATH_TO_LABELS = 'stockout_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

IMAGE_SIZE = (24, 18)

In [ ]:
img = Image.open(img_data)
image_np = load_image_into_numpy_array(img)
output_dict = json.loads(r.json())

In [ ]:
vis_util.visualize_boxes_and_labels_on_image_array(
  image_np,
  np.array(output_dict['detection_boxes']),
  np.array(output_dict['detection_classes']),
  np.array(output_dict['detection_scores']),
  category_index,
  instance_masks=output_dict.get('detection_masks'),
  use_normalized_coordinates=True,
  line_thickness=4)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)

In [ ]:
container.stop()

In [ ]:
# remove stopped container
!docker system prune -f